In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html
     |████████████████████████████████| 37.5 MB 129 kB/s 
     |████████████████████████████████| 190 kB 4.1 MB/s 
Cloning into 'mmdetection'...
remote: Enumerating objects: 24362, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 24362 (delta 3), reused 5 (delta 0), pack-reused 24348
Receiving objects: 100% (24362/24362), 37.49 MiB | 12.38 MiB/s, done.
Resolving deltas: 100% (17051/17051), done.
running install
running bdist_egg
running egg_info
creating mmdet.egg-info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
writing manifest file 'mmdet.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mmdet.egg-info/SOURCES.t

In [ ]:
# 런타임 재시작 필요

from mmdet.apis import init_detector, inference_detector
import mmcv

In [ ]:
%cd /content

/content


In [ ]:
%pip install wandb

     |████████████████████████████████| 1.8 MB 4.2 MB/s 
     |████████████████████████████████| 144 kB 88.2 MB/s 
     |████████████████████████████████| 181 kB 84.7 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=28eeadd7c67295aa42d41e906053050a5ab763d6661660e20f4a42a06228b95f
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
# Log in to your W&B account
wandb.login()

wandb: Currently logged in as: bglepark. Use `wandb login --relogin` to force relogin


True

In [ ]:
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset

@DATASETS.register_module(force=True)
class fishdataset(CocoDataset):
  CLASSES = ('Olive flounder', 'Korea rockfish', 'Red seabream' , 'Black porgy' , 'Rock bream') 

In [ ]:
config_file = '/content/mmdetection/configs/ssd/ssd300_coco.py'

checkpoint_file = '/content/mmdetection/checkpoints/ssd300_coco_20210803_015428-d231a06e.pth'


# config_file = '/content/mmdetection/configs/ssd/ssd300_coco.py'
# checkpoint_file = '/content/mmdetection/checkpoints/ssd300_coco_20210803_015428-d231a06e.pth'


In [ ]:
!cd /content/mmdetection; mkdir checkpoints
!wget -O /content/mmdetection/checkpoints/ssd300_coco_20210803_015428-d231a06e.pth https://download.openmmlab.com/mmdetection/v2.0/ssd/ssd300_coco/ssd300_coco_20210803_015428-d231a06e.pth

--2022-05-09 06:43:39--  https://download.openmmlab.com/mmdetection/v2.0/ssd/ssd300_coco/ssd300_coco_20210803_015428-d231a06e.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.18
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137242646 (131M) [application/octet-stream]
Saving to: ‘/content/mmdetection/checkpoints/ssd300_coco_20210803_015428-d231a06e.pth’

/content/mmdetectio 100%[===================>] 130.88M  12.4MB/s    in 10s     

2022-05-09 06:43:49 (12.8 MB/s) - ‘/content/mmdetection/checkpoints/ssd300_coco_20210803_015428-d231a06e.pth’ saved [137242646/137242646]



In [ ]:
!ls -lia /content/mmdetection/checkpoints

total 134036
5247027 drwxr-xr-x  2 root root      4096 May  9 06:43 .
5243024 drwxr-xr-x 19 root root      4096 May  9 06:43 ..
5247040 -rw-r--r--  1 root root 137242646 Nov  2  2021 ssd300_coco_20210803_015428-d231a06e.pth


In [ ]:
from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg.pretty_text)

input_size = 300
model = dict(
    type='SingleStageDetector',
    backbone=dict(
        type='SSDVGG',
        depth=16,
        with_last_pool=False,
        ceil_mode=True,
        out_indices=(3, 4),
        out_feature_indices=(22, 34),
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://vgg16_caffe')),
    neck=dict(
        type='SSDNeck',
        in_channels=(512, 1024),
        out_channels=(512, 1024, 512, 256, 256, 256),
        level_strides=(2, 2, 1, 1),
        level_paddings=(1, 1, 0, 0),
        l2_norm_scale=20),
    bbox_head=dict(
        type='SSDHead',
        in_channels=(512, 1024, 512, 256, 256, 256),
        num_classes=5,
        anchor_generator=dict(
            type='SSDAnchorGenerator',
            scale_major=False,
            input_size=300,
            basesize_ratio_range=(0.15, 0.9),
            strides=[8, 16, 32, 64, 100, 300],
            ratios=[[2], [2, 3], [2, 3], [2, 3], [2], [2]]),
        bbox_coder=dict(
        

In [ ]:
from mmdet.apis import set_random_seed

# dataset에 대한 환경 파라미터 수정. 
# cfg.dataset_type = 'fishdataset'
# cfg.data_root = '/content/drive/MyDrive/융합프로젝트_낚시/fish_sample_400_100_100_mmdetection/fish_sample/'

# # # train, val, test dataset에 대한 type, data_root, ann_file, img_prefix 환경 파라미터 수정. 
# cfg.data.train.type = 'fishdataset'
# cfg.data.train.data_root = '/content/drive/MyDrive/융합프로젝트_낚시/fish_sample_400_100_100_mmdetection/fish_sample/'
# cfg.data.train.dataset.ann_file = 'train.json'
# cfg.data.train.dataset.img_prefix = 'images'

# cfg.data.val.type = 'fishdataset'
# cfg.data.val.data_root = '/content/drive/MyDrive/융합프로젝트_낚시/fish_sample_400_100_100_mmdetection/fish_sample/'
# cfg.data.val.dataset.ann_file = 'valid.json'
# cfg.data.val.dataset.img_prefix = 'images'

# cfg.data.test.type = 'fishdataset'
# cfg.data.test.dataset.data_root = '/content/drive/MyDrive/융합프로젝트_낚시/fish_sample_400_100_100_mmdetection/fish_sample/'
# cfg.data.test.dataset.ann_file = 'test.json'
# cfg.data.test.dataset.img_prefix = 'images'

# class의 갯수 수정. 
# cfg.model.bbox_head.num_classes = 5 -> code내에서 수정
# pretrained 모델
cfg.load_from = '/content/mmdetection/checkpoints/ssd300_coco_20210803_015428-d231a06e.pth'

# 학습 weight 파일로 로그를 저장하기 위한 디렉토리 설정. 
cfg.work_dir = './tutorial_exps'

# # 학습율 변경 환경 파라미터 설정. 
# cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# CocoDataset의 경우 metric을 bbox로 설정해야 함.(mAP아님. bbox로 설정하면 mAP를 iou threshold를 0.5 ~ 0.95까지 변경하면서 측정)
cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 1
cfg.checkpoint_config.interval = 1

# # 두번 config를 로드하면 lr_config의 policy가 사라지는 오류로 인하여 설정. 
# cfg.lr_config.policy='step'
# # Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# cfg.runner.max_epochs = 10

# cfg.device='cuda'

cfg.data.samples_per_gpu = 4



cfg.device='cuda'

# cfg.train_pipeline[4].img_scale = (640,640)
# cfg.test_pipeline[1].img_scale = (640,640)
# cfg.data.train.dataset.pipeline[4].img_scale = (640,640)
# cfg.data.val.pipeline[1].img_scale = (640,640)
# cfg.data.test.pipeline[1].img_scale = (640,640)

In [ ]:
print(cfg.pretty_text)

input_size = 300
model = dict(
    type='SingleStageDetector',
    backbone=dict(
        type='SSDVGG',
        depth=16,
        with_last_pool=False,
        ceil_mode=True,
        out_indices=(3, 4),
        out_feature_indices=(22, 34),
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://vgg16_caffe')),
    neck=dict(
        type='SSDNeck',
        in_channels=(512, 1024),
        out_channels=(512, 1024, 512, 256, 256, 256),
        level_strides=(2, 2, 1, 1),
        level_paddings=(1, 1, 0, 0),
        l2_norm_scale=20),
    bbox_head=dict(
        type='SSDHead',
        in_channels=(512, 1024, 512, 256, 256, 256),
        num_classes=5,
        anchor_generator=dict(
            type='SSDAnchorGenerator',
            scale_major=False,
            input_size=300,
            basesize_ratio_range=(0.15, 0.9),
            strides=[8, 16, 32, 64, 100, 300],
            ratios=[[2], [2, 3], [2, 3], [2, 3], [2], [2]]),
        bbox_coder=dict(
        

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# train용 Dataset 생성. 
datasets = [build_dataset(cfg.data.train.dataset)]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
print(datasets[0])
# datasets[0].__dict__ 로 모든 self variables의 key와 value값을 볼 수 있음. 
datasets[0].__dict__.keys()


fishdataset Train dataset with number of images 400, and instance counts: 
+--------------------+-------+--------------------+-------+------------------+-------+-----------------+-------+----------------+-------+
| category           | count | category           | count | category         | count | category        | count | category       | count |
+--------------------+-------+--------------------+-------+------------------+-------+-----------------+-------+----------------+-------+
| 0 [Olive flounder] | 80    | 1 [Korea rockfish] | 80    | 2 [Red seabream] | 80    | 3 [Black porgy] | 80    | 4 [Rock bream] | 80    |
+--------------------+-------+--------------------+-------+------------------+-------+-----------------+-------+----------------+-------+


dict_keys(['ann_file', 'data_root', 'img_prefix', 'seg_prefix', 'proposal_file', 'test_mode', 'filter_empty_gt', 'file_client', 'CLASSES', 'coco', 'cat_ids', 'cat2label', 'img_ids', 'data_infos', 'proposals', 'flag', 'pipeline'])

In [ ]:
datasets

[
 fishdataset Train dataset with number of images 400, and instance counts: 
 +--------------------+-------+--------------------+-------+------------------+-------+-----------------+-------+----------------+-------+
 | category           | count | category           | count | category         | count | category        | count | category       | count |
 +--------------------+-------+--------------------+-------+------------------+-------+-----------------+-------+----------------+-------+
 | 0 [Olive flounder] | 80    | 1 [Korea rockfish] | 80    | 2 [Red seabream] | 80    | 3 [Black porgy] | 80    | 4 [Rock bream] | 80    |
 +--------------------+-------+--------------------+-------+------------------+-------+-----------------+-------+----------------+-------+]

In [ ]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

model.CLASSES = datasets[0].CLASSES
print(model.CLASSES)

('Olive flounder', 'Korea rockfish', 'Red seabream', 'Black porgy', 'Rock bream')


In [ ]:
import os.path as osp
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# epochs는 config의 runner 파라미터로 지정됨. 기본 12회 

train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-05-09 06:57:48,073 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-05-09 06:57:48,079 - mmdet - INFO - load checkpoint from local path: /content/mmdetection/checkpoints/ssd300_coco_20210803_015428-d231a06e.pth
2022-05-09 06:57:48,165 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.cls_convs.0.0.weight: copying a param with shape torch.Size([324, 512, 3, 3]) from checkpoint, the shape in current model is torch.Size([24, 512, 3, 3]).
size mismatch for bbox_head.cls_convs.0.0.bias: copying a param with shape torch.Size([324]) from checkpoint, the shape in current model is torch.Size([24]).
size mismatch for bbox_head.cls_convs.1.0.weight: copying a param with shape torch.Size([486, 1024, 3, 3]) from checkpoint, the shape in current model is torch.Size([36, 1024, 3, 3]).
size mismatch for bbox_head.cls_convs.1.0.bias: copying a param with shape torch.Size([486]) from checkpoint, the shape in current

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▅▃▃▃▂▂▂▁▂▂▂▂▁▁▁
train/loss_bbox,▅▆█▆▆▆▃▄▃▂▂▂▃▃▂▂▁
train/loss_cls,█▅▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁
val/bbox_mAP,▁
val/bbox_mAP_50,▁
val/bbox_mAP_75,▁
val/bbox_mAP_l,▁
val/bbox_mAP_m,▁
val/bbox_mAP_s,▁


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
2022-05-09 06:58:04,885 - mmdet - INFO - Epoch [1][10/100]	lr: 2.000e-03, eta: 1:50:52, time: 0.666, data_time: 0.578, memory: 1182, loss_cls: 8.5971, loss_bbox: 1.4972, loss: 10.0944
2022-05-09 06:58:07,954 - mmdet - INFO - Epoch [1][20/100]	lr: 2.000e-03, eta: 1:20:52, time: 0.307, data_time: 0.225, memory: 1182, loss_cls: 6.4384, loss_bbox: 1.7160, loss: 8.1544
2022-05-09 06:58:11,201 - mmdet - INFO - Epoch [1][30/100]	lr: 2.000e-03, eta: 1:11:52, time: 0.325, data_time: 0.237, memory: 1182, loss_cls: 5.4549, loss_bbox: 2.1005, loss: 7.5554
2022-05-09 06:58:15,243 - mmdet - INFO - Epoch [1][40/100]	lr: 2.000e-03, eta: 1:10:37, time: 0.404, data_time: 0.312, memory: 1182, loss_cls: 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100/100, 36.7 task/s, elapsed: 3s, ETA:     0s

...

2022-05-09 08:00:00,540 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.06s).
Accumulating evaluation results...


2022-05-09 08:00:00,642 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.719
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.868
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.831
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.719
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.807
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.807
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.807
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=

DONE (t=0.03s).


learning_rate,███████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▅▅▅▄▂▂▂▁▁▁▂▂▁▁▁▂▁▂▂▂▁▂▁▂▂▂▁▁▁▁▂▁▂▂▁▁▁▁
train/loss_bbox,█▅▅▅▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▁▂▂▁▁▁▁▁▂▁▂▁▁▁▁▁
train/loss_cls,█▇▆▅▅▄▂▂▃▁▁▁▂▂▁▂▁▂▁▂▂▂▁▂▁▂▂▂▁▁▁▁▂▁▂▂▁▁▂▁
val/bbox_mAP,▁▁▃▃▄▄▅▇▇███████████████████████████████
val/bbox_mAP_50,▁▂▅▄▆▆▇█████████████████████████████████
val/bbox_mAP_75,▁▁▃▄▄▅▆▇████████████████████████████████
val/bbox_mAP_l,▁▁▃▃▄▄▅▇▇███████████████████████████████
val/bbox_mAP_m,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/bbox_mAP_s,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
